<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Breast_Diagnosis_Model_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# !pip uninstall -y pandas tensorflow
# !pip install pandas tensorflow

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from PIL import Image
import itertools
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define your data directory path
data_dir = '/content/drive/My Drive/data2'

# Define possible resolution options and corresponding folder names
resolutionOptions = {
    '32x32': (32, 32),
    '56x56': (56, 56),
    '128x128': (128, 128),
    '256x256': (256, 256),
    '512x512': (512, 512)
}

# Ask the user to select a resolution
print("\nSelect the resolution:")
for i, (res_name, res_size) in enumerate(resolutionOptions.items(), 1):
    print(f"{i}. {res_name}")
print("")

selectionIndex = int(input("Enter the number corresponding to your choice: ")) - 1
if selectionIndex < 0 or selectionIndex >= len(resolutionOptions):
    raise ValueError("Invalid selection. Exiting.")

# Construct the data directory path and resolution based on the selected resolution
selectedResolutionName = list(resolutionOptions.keys())[selectionIndex]
selectedResolutionSize = resolutionOptions[selectedResolutionName]
data_dir = os.path.join('/content/drive/My Drive/data2', selectedResolutionName)

# Display the selected directory path and resolution
print(f"\nSelected data directory: {data_dir}")
print(f"Selected resolution: {selectedResolutionSize}\n")

# Check if the data directory exists
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Directory '{data_dir}' does not exist.")

# Get the list of class names (subfolders)
classes = [cls for cls in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, cls))]

for cls in classes:
    cls_dir = os.path.join(data_dir, cls)
    num_images = len(os.listdir(cls_dir))
    print(f"Class '{cls}': {num_images} images")

# Function to build and compile the CNN model
def build_compile_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])  # Ensure 'accuracy' is used
    return model

# Function to load a single image
def load_image(directory, cls, img_file):
    img_path = os.path.join(directory, cls, img_file)
    img = Image.open(img_path)
    return img

# Prepare data
X, y = [], []
label_map = {cls: idx for idx, cls in enumerate(classes)}

for cls in classes:
    cls_dir = os.path.join(data_dir, cls)
    img_files = os.listdir(cls_dir)

    for img_file in img_files:
        img_path = os.path.join(cls_dir, img_file)
        img = load_img(img_path, target_size=selectedResolutionSize)
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(label_map[cls])

X = np.array(X).astype('float32') / 255.0
y = np.array(y).astype('int')

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=22)

input_shape = (selectedResolutionSize[0], selectedResolutionSize[1], 3)
num_classes = len(classes)

# Build and compile model
model = build_compile_model(input_shape, num_classes)

# Train the model with validation split
history = model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))

# Predict and evaluate
y_train_pred = np.argmax(model.predict(X_train), axis=1)
y_test_pred = np.argmax(model.predict(X_test), axis=1)

cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

report_train = classification_report(y_train, y_train_pred, target_names=classes)
report_test = classification_report(y_test, y_test_pred, target_names=classes)

# Save the CNN architecture as a PNG figure
plot_model(model, to_file=f'model_architecture_{selectedResolutionSize[0]}x{selectedResolutionSize[1]}.png', show_shapes=True, show_layer_names=True)

# Draw a sample image (one image from each class)
fig, axes = plt.subplots(1, len(classes), figsize=(10, 10))
for ax, cls in zip(axes, classes):
    img_file = random.choice(os.listdir(os.path.join(data_dir, cls)))
    img = load_image(data_dir, cls, img_file)
    ax.imshow(img)
    ax.set_title(cls)
    ax.axis('off')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Select the resolution:
1. 32x32
2. 56x56
3. 128x128
4. 256x256
5. 512x512

Enter the number corresponding to your choice: 1

Selected data directory: /content/drive/My Drive/data2/32x32
Selected resolution: (32, 32)

Class 'benign': 437 images
Class 'malignant': 210 images
Class 'normal': 133 images
Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Exception ignored in: <function AtomicFunction.__del__ at 0x7d36f0841630>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py", line 302, in __del__
    self._bound_context.remove_function(self.name)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/context.py", line 1530, in remove_function
    pywrap_tfe.TFE_ContextRemoveFunction(self._handle, name)
KeyboardInterrupt: 


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.5575 - loss: 0.9905 - val_accuracy: 0.5577 - val_loss: 0.9458
Epoch 2/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5817 - loss: 0.9397 - val_accuracy: 0.6090 - val_loss: 0.9035
Epoch 3/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.5977 - loss: 0.9045 - val_accuracy: 0.6410 - val_loss: 0.8523
Epoch 4/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6275 - loss: 0.8763 - val_accuracy: 0.6346 - val_loss: 0.8373
Epoch 5/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6443 - loss: 0.8730 - val_accuracy: 0.6603 - val_loss: 0.8095
Epoch 6/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6301 - loss: 0.8174 - val_accuracy: 0.6795 - val_loss: 0.7864
Epoch 7/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6487 - loss: 0.7678 - val_accuracy: 0.6667 - val_loss: 0.7636
Epoch 8/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.6505 - loss: 0.7507 - val_accuracy: 0.6667 - val_loss: 0.

In [ ]:
# Save the figure
plt.savefig(f'sample_images_{selectedResolutionSize[0]}x{selectedResolutionSize[1]}.png')
plt.close()

# Save confusion matrices
plot_confusion_matrices(cm_train, cm_test, classes,
                        train_title='Training Confusion Matrix',
                        test_title='Testing Confusion Matrix',
                        filename=f'confusion_matrices_{selectedResolutionSize[0]}x{selectedResolutionSize[1]}.png',
                        font_size=8)  # Reduce font size for clarity

# Plot convergence curves (accuracy and loss in one figure as subplots) and save as PNG
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 5))  # Adjusted figsize

# Plot metrics
def plot_metric(metric_name, ax, title, ylabel):
    if metric_name in history.history:
        ax.plot(history.history[metric_name], label='Train ' + title)
    if f'val_{metric_name}' in history.history:
        ax.plot(history.history[f'val_{metric_name}'], label='Validation ' + title)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.set_title(f'{title} - {selectedResolutionName}')
    ax.grid()

# Accuracy plot
plot_metric('accuracy', ax1, 'Accuracy', 'Accuracy')

# Loss plot
plot_metric('loss', ax2, 'Loss', 'Loss')

plt.tight_layout()
plt.savefig(f'convergence_curve_{selectedResolutionName}.png')
plt.show()

In [ ]:
# Initialize a list to store metrics for saving
metrics = []

# After training and predictions
train_predictions = np.argmax(model.predict(X_train), axis=1)
test_predictions = np.argmax(model.predict(X_test), axis=1)

# Compute metrics for training data
train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions, average='weighted')
train_recall = recall_score(y_train, train_predictions, average='weighted')
train_f1 = f1_score(y_train, train_predictions, average='weighted')

# Store training metrics in the list
metrics.append({
    'Dataset': 'Training',
    'Resolution': f'{selectedResolutionSize[0]}x{selectedResolutionSize[1]}',
    'Accuracy': train_accuracy,
    'Precision': train_precision,
    'Recall': train_recall,
    'F1 Score': train_f1
})

# Compute metrics for testing data
test_accuracy = accuracy_score(y_test, test_predictions)
test_precision = precision_score(y_test, test_predictions, average='weighted')
test_recall = recall_score(y_test, test_predictions, average='weighted')
test_f1 = f1_score(y_test, test_predictions, average='weighted')

# Store testing metrics in the list
metrics.append({
    'Dataset': 'Testing',
    'Resolution': f'{selectedResolutionSize[0]}x{selectedResolutionSize[1]}',
    'Accuracy': test_accuracy,
    'Precision': test_precision,
    'Recall': test_recall,
    'F1 Score': test_f1
})

# Create a DataFrame from the metrics list
metrics_df = pd.DataFrame(metrics)

# Display metrics DataFrame
print(metrics_df)

# Define filename with resolution
excel_filename = f'performance_metrics_{selectedResolutionSize[0]}x{selectedResolutionSize[1]}.xlsx'

# Save the DataFrame to an Excel file
metrics_df.to_excel(excel_filename, index=False, sheet_name='Performance Metrics')

print(f"Metrics saved to {excel_filename}")
